# Replication with Actual Data

This notebook replicates the paper's econometric analysis using the **actual** panel dataset built from FIFA rankings, club rankings, and World Development Indicators.

**Data Sources**:
- FIFA World Rankings (1993-2010)
- FIFA Club Rankings (2000-2010)
- World Bank WDI (1993-2010)

**Output**: Regression results for equations (1), (2), and (3) saved to CSV files.

**Project docs**: [README.md](../README.md), [Data Dictionary](../docs/data-dictionary.md), [Data Pipeline](../docs/data-pipeline.md)

In [ ]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from IPython.display import display
from linearmodels.panel import PanelOLS
from linearmodels.iv import IV2SLS

print(f"Python: {sys.version}")
print(f"Pandas: {pd.__version__}")
print(f"Working directory: {Path.cwd()}")

## Load Actual Panel Data

Load the panel dataset built from FIFA points, club rankings, and WDI data (1993-2010).

In [ ]:
# Load the panel dataset
data_path = Path("../data/analysis/panel.csv")

if not data_path.exists():
    raise FileNotFoundError(f"Panel data not found at {data_path}. Run: python3 scripts/build_panel.py")

df = pd.read_csv(data_path)
print(f"Loaded {len(df)} observations")
print(f"Countries: {df['country'].nunique()}")
print(f"Years: {df['year'].min()}-{df['year'].max()}")
print(f"Variables: {list(df.columns)}")
print(f"\nConfederations:
{df['confed'].value_counts()}")

### Data Snapshot

In [ ]:
# Display first few rows and summary stats
display(df.head(10))
print(f"\nDataset shape: {df.shape}")

# Check for missing values
missing = df.isnull().sum()
print(f"\nMissing values:
{missing[missing > 0]}")

## Load Model Runner Helpers

Import the regression functions from the replication script.

In [ ]:
# Add repo root to path
repo_root = Path.cwd().parent
sys.path.append(str(repo_root))

try:
    from scripts.replicate_stata import fit_fe_ols, fit_fe_iv
    print("Successfully imported fit_fe_ols and fit_fe_iv from scripts.replicate_stata")
except Exception as e:
    print(f"Import failed: {e}")
    print("Using fallback implementations...")
    
    def two_way_demean(df, cols, entity, time):
        out = df.copy()
        for col in cols:
            overall = out[col].mean()
            ent_mean = out.groupby(entity)[col].transform("mean")
            time_mean = out.groupby(time)[col].transform("mean")
            out[col] = out[col] - ent_mean - time_mean + overall
        return out

    def _drop_absorbed(panel, exog):
        names = list(panel.index.names)
        if len(names) < 2 or names[0] is None or names[1] is None:
            return exog, []
        entity, time = names[0], names[1]
        work = panel.reset_index()
        demeaned = two_way_demean(work, exog, entity, time)
        dropped = []
        keep = []
        for col in exog:
            if demeaned[col].var() <= 1e-12:
                dropped.append(col)
            else:
                keep.append(col)
        return keep, dropped

    def fit_fe_ols(panel, dep, exog):
        keep, dropped = _drop_absorbed(panel, exog)
        if dropped:
            print(f"Dropped absorbed variables: {', '.join(dropped)}")
        if not keep:
            raise ValueError("All regressors absorbed by fixed effects.")
        y = panel[dep]
        X = panel[keep]
        model = PanelOLS(y, X, entity_effects=True, time_effects=True)
        return model.fit(cov_type="clustered", cluster_entity=True, debiased=True)

    def fit_fe_iv(df, dep, exog, endog, instr, entity, time):
        cols = [dep] + exog + [endog] + instr + [entity, time]
        work = df[cols].dropna().copy()
        work = two_way_demean(work, [dep] + exog + [endog] + instr, entity, time)
        keep_exog = [c for c in exog if work[c].var() > 1e-12]
        if not keep_exog:
            raise ValueError("All exogenous regressors absorbed.")
        endog_v = work[endog]
        if endog_v.var() <= 1e-12:
            raise ValueError("Endogenous regressor absorbed.")
        keep_instr = [c for c in instr if work[c].var() > 1e-12]
        if not keep_instr:
            raise ValueError("All instruments absorbed.")
        y = work[dep]
        X = work[keep_exog]
        Z = work[keep_instr]
        model = IV2SLS(y, X, endog_v, Z)
        return model.fit(cov_type="clustered", clusters=work[entity], debiased=True)

## Equation (1): Baseline Model

**Specification**: FIFA points ~ GDP per capita + GDP per capita² + Population + Population²

**Estimation**: Fixed-effects OLS with two-way (country + time) effects and clustered standard errors

In [ ]:
# Define variables
dep = "fifa_points"
base = ["gdp_pc", "gdp_pc_sq", "pop", "pop_sq"]

# Prepare panel
panel = df.set_index(["country", "year"])

# Run Equation (1)
print("="*80)
print("EQUATION (1): Baseline Model")
print("="*80)
res1 = fit_fe_ols(panel, dep, base)
print(res1.summary)
display(res1.summary)

## Equation (2): Model with Macro/Resource Controls

**Specification**: Equation (1) + Trade + Inflation + Oil rents + Life expectancy

**Estimation**: Fixed-effects OLS with two-way effects and clustered SEs

In [ ]:
# Define additional controls
macro = ["trade", "infl", "oil", "leb"]

# Run Equation (2)
print("="*80)
print("EQUATION (2): Model with Controls")
print("="*80)
res2 = fit_fe_ols(panel, dep, base + macro)
print(res2.summary)
display(res2.summary)

## Equation (3): IV/2SLS with Club Strength

**Specification**: Equation (2) + Club strength (instrumented by urban population + urban population²)

**Estimation**: Fixed-effects IV/2SLS with clustered SEs

**Instruments**: urbpop, urbpop_sq (for endogenous "club" variable)

In [ ]:
# Define IV variables
club = "club"
instr = ["urbpop", "urbpop_sq"]

# Check club data availability
club_available = df[club].notna().sum()
club_total = len(df)
print(f"\nClub data availability: {club_available}/{club_total} ({100*club_available/club_total:.1f}%)")

# Run Equation (3) if we have club data
if club_available > 0:
    print("="*80)
    print("EQUATION (3): IV/2SLS with Club Strength")
    print("="*80)
    try:
        res3 = fit_fe_iv(df, dep, base + macro, club, instr, "country", "year")
        print(res3.summary)
        display(res3.summary)
    except Exception as e:
        print(f"Error running IV regression: {e}")
        print("\nThis may occur if:")
        print("- All instruments are absorbed by fixed effects")
        print("- No variation in instruments after demeaning")
else:
    print("No club data available - skipping Equation (3)")

## Confederation-Level Analysis

Run regressions separately for each football confederation.

In [ ]:
# Run by confederation
if "confed" in df.columns:
    confederations = sorted(df["confed"].dropna().unique())
    
    for confed in confederations:
        subset = df[df["confed"] == confed]
        if subset.empty:
            continue
        
        print(f"\n{'='*60}")
        print(f"CONFEDERATION: {confed}")
        print(f"Observations: {len(subset)}")
        print(f"Countries: {subset['country'].nunique()}")
        print(f"{'='*60}")
        
        panel_sub = subset.set_index(["country", "year"])
        
        # Equation (1)
        try:
            res1c = fit_fe_ols(panel_sub, dep, base)
            print(f"\nEquation (1) - {confed}")
            print(res1c.summary.tables[1])
        except Exception as e:
            print(f"\nEquation (1) failed for {confed}: {e}")
        
        # Equation (2)
        try:
            res2c = fit_fe_ols(panel_sub, dep, base + macro)
            print(f"\nEquation (2) - {confed}")
            print(res2c.summary.tables[1])
        except Exception as e:
            print(f"\nEquation (2) failed for {confed}: {e}")
        
        # Equation (3) - typically only for UEFA
        if confed.upper() == "UEFA" and club_available > 0:
            try:
                res3c = fit_fe_iv(subset, dep, base + macro, club, instr, "country", "year")
                print(f"\nEquation (3) - {confed}")
                print(res3c.summary.tables[1])
            except Exception as e:
                print(f"\nEquation (3) failed for {confed}: {e}")

## Export Results

Save regression results to CSV files for further analysis.

In [ ]:
# Create results directory
results_dir = Path("results")
results_dir.mkdir(exist_ok=True)

def save_summary(res, path):
    """Save regression summary to CSV."""
    table = res.summary.tables[1].as_csv()
    with open(path, "w") as f:
        f.write(table)
    print(f"Saved: {path}")

# Save full sample results
try:
    save_summary(res1, results_dir / "eq1_full.csv")
    save_summary(res2, results_dir / "eq2_full.csv")
    if 'res3' in dir():
        save_summary(res3, results_dir / "eq3_full.csv")
except Exception as e:
    print(f"Error saving results: {e}")

print(f"\nResults saved to {results_dir}/")
print(f"Files: {list(results_dir.glob('*.csv'))}")

## Summary and Comparison

### Key Findings from Actual Data

Compare these results to the synthetic data notebook and eventually to the original paper.

In [ ]:
# Summary statistics
print("DATASET SUMMARY")
print("="*60)
print(f"Total observations: {len(df)}")
print(f"Countries: {df['country'].nunique()}")
print(f"Year range: {df['year'].min()}-{df['year'].max()}")
print(f"Balanced panel: {len(df) / (df['country'].nunique() * df['year'].nunique()):.1%} complete")

print("\nCONFEDERATION DISTRIBUTION:")
print(df['confed'].value_counts())

print("\nDEPENDENT VARIABLE (fifa_points):")
print(df['fifa_points'].describe())

print("\nKEY INDEPENDENT VARIABLES:")
for var in ['gdp_pc', 'pop', 'club', 'urbpop']:
    if var in df.columns:
        missing = df[var].isnull().sum()
        print(f"  {var}: {missing}/{len(df)} missing ({100*missing/len(df):.1f}%)")

## Next Steps

### Validation Tasks
- [ ] Compare coefficients to original paper
- [ ] Verify sign and significance match expectations
- [ ] Check standard error magnitudes
- [ ] Validate R-squared values

### Data Quality Checks
- [ ] Investigate countries with missing club data
- [ ] Verify country name mappings
- [ ] Check for outliers in FIFA points
- [ ] Validate WDI data ranges

### Extensions
- [ ] Run placebo tests
- [ ] Test robustness to different time periods
- [ ] Add additional controls
- [ ] Export formatted tables

### For Comparison to Paper
1. Locate original Stata output files (if available)
2. Create coefficient comparison table
3. Document any discrepancies
4. Investigate sources of difference (Stata 8/9 defaults, data revisions, etc.)